In [34]:
import pyautogui
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import win32api
import win32con
import time
import numpy as np
import os

import torch.nn as nn
import torch.nn.functional as F

In [58]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=5):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        # Modify fc1 to match the saved model dimensions
        self.fc1 = nn.Linear(64 * 16 * 16, 128)  # Changed to [128, 16384]
        self.fc2 = nn.Linear(128, num_classes)   # Changed to [5, 128]
        
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 16 * 16)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  # Apply Dropout
        x = self.fc2(x)
        return x


In [59]:

model = SimpleCNN(5)

# Load the model state
model.load_state_dict(torch.load('D:\subway\subway_surfer_model.pth', map_location='cpu'))  # Use 'cpu' or 'cuda'
model.eval()



C:\Users\ABHIRAM\AppData\Local\Temp\ipykernel_8952\1688527614.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('D:\subway\subway_surfer_m

SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=16384, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=5, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [60]:

from PIL import Image
import torch
from torchvision import transforms

# Preprocessing function for the model
def preprocess_image(image, device='cpu'):
    # Check if the image is a file path or PIL Image
    if isinstance(image, str):
        image = Image.open(image)  # Open the image if it's a file path

    # Define transformation pipeline
    transform = transforms.Compose([
        transforms.RandomResizedCrop(128),  # Resize images to a smaller size
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Apply transformation and add batch dimension
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension and move to device (CPU/GPU)

    return image


# Function to press a key
def press_key(key_code):
    win32api.keybd_event(key_code, 0, 0, 0)  # Key down
    time.sleep(0.1)  # Hold the key for a short duration
    win32api.keybd_event(key_code, 0, win32con.KEYEVENTF_KEYUP, 0)  # Key up

# Key codes for controls
key_codes = {
    'up': 0x26,    # Up arrow
    'down': 0x28,  # Down arrow
    'left': 0x25,  # Left arrow
    'right': 0x27, # Right arrow
}

In [61]:
classes = os.listdir(r'D:\subway\SUBWAY\SUBWAY')
classes

['down', 'left', 'noop', 'right', 'up']

In [62]:
#hot-key
stop_flag = False

# Function to stop capturing when 'q' is pressed
def stop_capture():
    global stop_flag
    stop_flag = True

# Add a hotkey to stop when 'q' is pressed
keyboard.add_hotkey('q', stop_capture)


<function keyboard.add_hotkey.<locals>.remove_()>

In [63]:
lcount,rcount=0,0

In [64]:
import pyautogui
import torch
import time
import win32api
import win32con

In [65]:
classes = ['left', 'right', 'up', 'down', 'noop']  # Add actual class labels

In [66]:
def press_key(key_code):
    """Press and release a key given its virtual key code."""
    win32api.keybd_event(key_code, 0, 0, 0)  # Press the key
    time.sleep(0.1)  # Short delay for key press
    win32api.keybd_event(key_code, 0, win32con.KEYEVENTF_KEYUP, 0)  # Release the key

In [68]:
import time
import pyautogui
import win32api
import win32con
import torch
import keyboard

# Initialize variables
stop_flag = True  # Initialize the stop flag
current_lanes = ['left', 'center', 'right']  # Define possible lanes
lane = 1  # Start in the center lane (index 1)

# Key codes for controls
key_codes = {
    'up': 0x26,    # Up arrow
    'down': 0x28,  # Down arrow
    'left': 0x25,  # Left arrow
    'right': 0x27, # Right arrow
}

# Preprocessing function (assumed to be implemented)
# def preprocess_image(image): ...

while True:
    if win32api.GetAsyncKeyState(ord('S')):  # Start capturing when 'S' is pressed
        stop_flag = False  # Set the stop flag
        print("Started capturing")
        lane = 1  # Start in the center lane

    while not stop_flag:
        # Take a screenshot of the entire screen
        screenshot = pyautogui.screenshot()
        input_image = preprocess_image(screenshot)

        # Perform inference
        with torch.no_grad():
            output = model(input_image)

        # Get the predicted class
        _, predicted_class = output.max(1)
        action = classes[predicted_class.item()]

        if action in key_codes:
            # Handle left action
            if action == 'left':
                if current_lanes[lane] == 'left':
                    print("Already in the left lane; ignoring left action.")
                else:
                    lane -= 1  # Move left
                    lane = max(lane, 0)  # Ensure lane doesn't go out of bounds
                    press_key(key_codes[action])
                    time.sleep(0.1)
                    print(f"Moved left to lane {current_lanes[lane]}")

            # Handle right action
            elif action == 'right':
                if current_lanes[lane] == 'right':
                    print("Already in the right lane")
                else:
                    lane += 1  # Move right
                    lane = min(lane, 2)  # Ensure lane doesn't go out of bounds
                    time.sleep(0.2)
                    press_key(key_codes[action])
                    print(f"Moved right to lane {current_lanes[lane]}")

            # Handle up action
            elif action == 'up':
                time.sleep(0.1)
                press_key(key_codes[action])
                print("Jumped (up action)")

            # Handle down action
            elif action == 'down':
                press_key(key_codes[action])
                print("Rolled (down action)")

        # Check for no operation
        elif action == 'noop':
            print("No operation performed.")

        # Stop capturing if 'Q' is pressed
        if keyboard.is_pressed('q'):
            stop_flag = True
            print("Stopped capturing.")
            break

        # Delay to avoid high CPU usage
        time.sleep(0.1)

    time.sleep(0.1)  # Delay in the main loop


Started capturing
Rolled (down action)
Rolled (down action)
Rolled (down action)
Rolled (down action)
Rolled (down action)
Rolled (down action)
Rolled (down action)
Rolled (down action)
